In [1]:
!PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0

In [2]:
# !pip install auto_gptq==0.2.0
# !pip install transformers==4.35
# !pip install --upgrade optimum
!pip install --upgrade git+https://github.com/huggingface/transformers.git
# !pip install --upgrade accelerate

  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/pip-req-build-6qpojs05
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/pip-req-build-6qpojs05
^C
ERROR: Operation cancelled by user


In [6]:
from transformers import pipeline, Conversation, AutoTokenizer
from src.packages.utils.parameter_server import ParameterServer
import torch

parameter_server = ParameterServer()

base_model_name = "bardsai/jaskier-7b-dpo-v6.1"
generator = pipeline("conversational", model=base_model_name, torch_dtype=torch.half, device_map=parameter_server.settings.device)
model = generator.model
model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer

LlamaTokenizerFast(name_or_path='bardsai/jaskier-7b-dpo-v6.1', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
conversation = Conversation("Хто такий Дарт Вейдер?")
conversation = generator(conversation, pad_token_id=generator.tokenizer.eos_token_id)
print(conversation.messages[-1]["content"])

Дарт Вейдер, відомий також як Люк Скайвокер-Вейдер, є вигаданим персонажем зі всесвіту «Зоряних війн». Він постає як темний лицар-джедай, який став одним із головних антагоністів у оригінальній трилогії. Вейдер був асимільований Силою темними рицарями, аби стати Імперією їхнім інструментом, і відомий завдяки своїй чорній броні, поглинутій ненависті та зловісному диханню.


In [9]:
print(torch.backends.mps.is_available())

True


In [10]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 1,703,936 || all params: 7,243,436,032 || trainable%: 0.023523863432663224


/Users/rkovalch/miniconda3/envs/simulationllm/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [12]:
peft_model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=1024, bias=False)


In [75]:
from src.packages.constants.error_constants import ErrorConstants
from src.packages.prompts.instruction_tuning_gec_prompts import InstructionTuningGecPrompts
from ua_gec import Corpus
corpus = Corpus(partition="test", annotation_layer="gec-only")
for doc in corpus:
    print("\n---Source starts:---\n")
    print(doc.source[:512])         # "I likes it."
    print("\n---Source ends:---\n")
    print("\n---Target starts:---\n")
    print(doc.target[:512])         # "I like it."
    print("\n---Target ends---\n")
    print("\n---Annotation starts:---\n")
    print(str(doc.annotated)[:700])      # <AnnotatedText("I {likes=>like} it.")
    print(doc.meta.region)    # "Київська"
    print("\n---Annotation ends")
    print("\n---Prompt starts")
    prompt = InstructionTuningGecPrompts.PROMPT.format_prompt(
        query=doc.source[:512],
        error_types=ErrorConstants.ERROR_TYPES
    )
    print(prompt)
    break


---Source starts:---

Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік.
"Я не є доктор" - сказав я: "Чому вам не піти до лікаря?".
"Босс" - сказав він: "Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон. Він є єдиний лікар в місті і Масса банки сильно погано обурудувані. Він відправив мені, щоб спитав чи погоджуйтеся хоч прийти".
"Як чоловік до чоловіка" - 

---Source ends:---


---Target starts:---

Наступного ранку рівно об одинадцятій годині, коли я сидів сам, дядько Том шастав у готелі і попросив у лікаря підійти і побачити Джанге Банк, здавалось, це був майор і дуже хворий чоловік.
"Я не є доктор", — сказав я. — Чому вам не піти до лікаря?".
"Босс, — сказав він. — Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон. Він — єдиний лікар у місті і Масса банки сильно погано обурудувані. Він

In [19]:
len(tokenizer.tokenize(prompt.to_string()))

2860

In [70]:
prompt.to_string()

'Given ("SOURCE") text, correct errors if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language.\nUse following set of errors ("ERROR_TYPES"):\n[\'Fluency\', \'Grammar\', \'Punctuation\', \'Spelling\']\nIf you would detect error ("ERROR"), generate correction ("CORRECTION") following the structure and specifying error type ("ERROR_TYPE"):\n{("ERROR")=>("CORRECTION"):::("ERROR_TYPE")}\nOtherwise keep original text.\n\nSOURCE: Byte for France або “Мій досвід ведення блогу у Instagram”\nОстанні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.\n\nСьогодні розповім про те як і навіщо мене занесло у Instagram. Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – https://www.instagram.com/yevhenii_kanivets/\n\nМоє бачення Instagram\nКолись давно я прочитав статтю,\nERROR_TYPES: [\'Fluency\',

In [62]:
max_length = 256

model_inputs = tokenizer(prompt.to_string(), max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
model_inputs

{'input_ids': tensor([[    1, 12628,  4734, 17426,  1243,  2245, 28725,  4714,  7559,   513,
          2169, 28725, 16427,   288,   420,  8619,   325, 28777,  3212,  3479,
          7419,  3198, 19445, 28731, 10290,   354, 23129,   753, 15589, 28723,
            13,  8543,  2296,   808,   302,  7559,  4734,  4732, 28730,  3657,
         28735, 21021,    13,  1410,  3188, 28718,  2090,   647,   464, 28777,
          3212,  3479,   647,   464, 28753, 18181, 10223,   647,   464,  4941,
          3572,  1421,    13,  3381,   368,   682,  6705,  2118,  4734,  4732,
          3548,  8270, 22561,  4734, 28743,  1017,   896,  5324,  1243,  2296,
           272,  4693,   304,  1229,  6219,  2118,  1212,  4734,  4732, 28730,
          3657, 21021,    13, 28751,   618,  4732,  1243,  4929,   618, 28743,
          1017,   896,  5324,  1243,   564, 28747,   618,  4732, 28730,  3657,
          1243, 28752,    13, 15681,  3578,  1840,  3493,  2245, 28723,    13,
            13, 17426, 28747, 20376,  

In [66]:
response = peft_model.generate(
    input_ids=model_inputs["input_ids"].to("mps"),
    attention_mask=model_inputs["attention_mask"].to("mps"),
    max_new_tokens=100
)
response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1, 12628,  4734, 17426,  1243,  2245, 28725,  4714,  7559,   513,
          2169, 28725, 16427,   288,   420,  8619,   325, 28777,  3212,  3479,
          7419,  3198, 19445, 28731, 10290,   354, 23129,   753, 15589, 28723,
            13,  8543,  2296,   808,   302,  7559,  4734,  4732, 28730,  3657,
         28735, 21021,    13,  1410,  3188, 28718,  2090,   647,   464, 28777,
          3212,  3479,   647,   464, 28753, 18181, 10223,   647,   464,  4941,
          3572,  1421,    13,  3381,   368,   682,  6705,  2118,  4734,  4732,
          3548,  8270, 22561,  4734, 28743,  1017,   896,  5324,  1243,  2296,
           272,  4693,   304,  1229,  6219,  2118,  1212,  4734,  4732, 28730,
          3657, 21021,    13, 28751,   618,  4732,  1243,  4929,   618, 28743,
          1017,   896,  5324,  1243,   564, 28747,   618,  4732, 28730,  3657,
          1243, 28752,    13, 15681,  3578,  1840,  3493,  2245, 28723,    13,
            13, 17426, 28747, 20376,   354,  4843, 2

In [67]:
response.shape

torch.Size([1, 356])

In [69]:
tokenizer.batch_decode(response.detach().cpu().numpy(), skip_special_tokens=False)

['<s> Given ("SOURCE") text, correct errors if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language.\nUse following set of errors ("ERROR_TYPES"):\n[\'Fluency\', \'Grammar\', \'Punctuation\', \'Spelling\']\nIf you would detect error ("ERROR"), generate correction ("CORRECTION") following the structure and specifying error type ("ERROR_TYPE"):\n{("ERROR")=>("CORRECTION"):::("ERROR_TYPE")}\nOtherwise keep original text.\n\nSOURCE: Byte for France або “Мій досвід ведення блогу у Instagram”\nОстанні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.\n\nСьогодні розповім про те як і навіщо я почав вести блог у Instagram, а також про мої перші кроки, перешкоди, які я зустрів, а також про те, як я зміг перемогти над ними.\nERROR: Останні 3 місяці\nCORRECTION: Останні три місяці :::Spelling\nERROR: мого життя\nCORRECTI

In [24]:
peft_model("привіт")

device(type='mps', index=0)